# Fase 4: Entrenamiento y Selección de Modelos de Clasificación

## Objetivo del Notebook

Este notebook constituye la cuarta fase del pipeline de mantenimiento predictivo para sistemas de moto-compresores. El objetivo principal es entrenar, evaluar y seleccionar un modelo de clasificación binaria capaz de predecir si ocurrirá una falla en los próximos 7 días, utilizando las características derivadas del proceso de ingeniería de características implementado en la fase anterior.

### Metodología de Entrenamiento

La metodología implementada se basa en principios fundamentales de machine learning para series temporales, priorizando:

1. **División Cronológica de Datos**: Implementación de una división temporal que respete la naturaleza secuencial de los datos operacionales, evitando la fuga de información (data leakage) que comprometería la validez del modelo.

2. **Manejo de Desbalance de Clases**: Aplicación de técnicas específicas para abordar la distribución desigual entre muestras de operación normal y muestras pre-falla, fundamental en aplicaciones de mantenimiento predictivo.

3. **Evaluación Robusta**: Utilización de métricas de rendimiento apropiadas para clasificación desbalanceada, que proporcionen una evaluación objetiva de la capacidad predictiva del modelo.

4. **Simplicidad Computacional**: Selección de algoritmos que mantengan un balance óptimo entre rendimiento predictivo y eficiencia computacional.

### Librerías y Dependencias

El desarrollo requiere las siguientes librerías especializadas:
- **pandas, numpy**: Manipulación y procesamiento de datos
- **pathlib**: Gestión de rutas del sistema de archivos
- **joblib**: Serialización eficiente de modelos
- **matplotlib, seaborn**: Visualización de resultados y métricas
- **sklearn**: Algoritmos de machine learning, pipelines y métricas de evaluación

In [1]:
# Importación de librerías fundamentales para manipulación de datos
import pandas as pd
import numpy as np
from pathlib import Path
import joblib
import warnings
warnings.filterwarnings('ignore')

# Librerías para visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de estilo para visualizaciones
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Importación de algoritmos de machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# Importación de herramientas de pipeline y preprocesamiento
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Importación de métricas de evaluación para clasificación
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    roc_auc_score, 
    precision_recall_curve,
    roc_curve,
    auc,
    f1_score
)

# Definición de rutas del proyecto
data_processed_path = Path('./data/processed')
models_path = Path('./data/models')

# Creación de directorio de modelos si no existe
models_path.mkdir(exist_ok=True)

print("Configuración del entorno completada exitosamente")
print(f"Directorio de datos procesados: {data_processed_path}")
print(f"Directorio de modelos: {models_path}")

Configuración del entorno completada exitosamente
Directorio de datos procesados: data/processed
Directorio de modelos: data/models


## Paso 2: Carga y Preparación de Datos

### Proceso de Carga de Dataset

En esta etapa se procede a cargar el dataset resultante del proceso de ingeniería de características desarrollado en la fase anterior. El archivo `featured_dataset_for_modeling.parquet` contiene el conjunto completo de características derivadas, incluyendo:

- **Características originales de sensores**: Variables operacionales directas del moto-compresor
- **Características de ventanas móviles**: Estadísticos calculados sobre períodos temporales específicos
- **Características de lag temporal**: Variables retardadas que capturan dependencias temporales
- **Variable objetivo**: Indicador binario de proximidad a falla (ventana de 7 días)

### Separación de Características y Variable Objetivo

La preparación de datos requiere la separación clara entre la matriz de características (X) y el vector de variable objetivo (y). Esta separación es fundamental para el entrenamiento supervisado del modelo de clasificación, donde:

- **X**: Contiene todas las características derivadas que el modelo utilizará para realizar predicciones
- **y**: Contiene la variable binaria 'falla' que indica si la muestra está dentro del período de 7 días previo a una falla

In [3]:
# Carga del dataset con características de ingeniería
dataset_path = data_processed_path / 'featured_dataset_for_modeling.parquet'

try:
    df = pd.read_parquet(dataset_path)
    print(f"Dataset cargado exitosamente desde: {dataset_path}")
    print(f"Dimensiones del dataset: {df.shape}")
    print(f"Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo {dataset_path}")
    print("Asegúrese de haber ejecutado el notebook 03_feature_engineering.ipynb")
    raise

# Verificación de la estructura del dataset
print("\n=== Información General del Dataset ===")
print(df.info())

# Análisis de la variable objetivo
print("\n=== Distribución de la Variable Objetivo 'falla' ===")
target_distribution = df['falla'].value_counts().sort_index()
print(target_distribution)
print(f"\nPorcentaje de muestras normales (0): {(target_distribution[0] / len(df) * 100):.2f}%")
print(f"Porcentaje de muestras pre-falla (1): {(target_distribution[1] / len(df) * 100):.2f}%")
print(f"Ratio de desbalance: {target_distribution[0] / target_distribution[1]:.1f}:1")

# Separación de características y variable objetivo
print("\n=== Preparación de Matrices de Entrenamiento ===")

# Definición de la matriz de características (X)
# Excluimos la columna 'falla' ya que es nuestra variable objetivo
feature_columns = [col for col in df.columns if col != 'falla']
X = df[feature_columns].copy()

# Definición del vector objetivo (y)
y = df['falla'].copy()

print(f"Matriz de características (X): {X.shape}")
print(f"Vector objetivo (y): {y.shape}")
print(f"Número de características disponibles: {len(feature_columns)}")

# Verificación de valores faltantes en características
missing_values = X.isnull().sum().sum()
print(f"Valores faltantes en características: {missing_values}")

if missing_values > 0:
    print("\nCaracterísticas con valores faltantes:")
    missing_by_column = X.isnull().sum()
    print(missing_by_column[missing_by_column > 0])

Dataset cargado exitosamente desde: data/processed/featured_dataset_for_modeling.parquet
Dimensiones del dataset: (19752, 144)
Memoria utilizada: 11.00 MB

=== Información General del Dataset ===
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 19752 entries, 2023-01-11 00:00:00 to 2025-04-12 23:00:00
Columns: 144 entries, rpm to presion_descarga_diff_12H
dtypes: float32(144)
memory usage: 11.0 MB
None

=== Distribución de la Variable Objetivo 'falla' ===


KeyError: 'falla'

## Paso 3: División Cronológica de Datos (Time-Based Split)

### Importancia Crítica de la División Temporal

La división de datos en series temporales requiere un enfoque metodológicamente diferente al utilizado en problemas de clasificación estándar. La función `train_test_split` de scikit-learn con `shuffle=True` es **fundamentalmente incorrecta** para datos de series temporales por las siguientes razones:

#### Problemas de la División Aleatoria:

1. **Fuga de Información (Data Leakage)**: Una división aleatoria permite que el modelo acceda a información futura durante el entrenamiento, creando una ventaja artificial que no existiría en un escenario de predicción real.

2. **Validación No Realista**: En aplicaciones industriales de mantenimiento predictivo, el modelo debe predecir eventos futuros basándose únicamente en datos históricos. Una división aleatoria no simula esta condición operacional.

3. **Sobreestimación del Rendimiento**: Los resultados obtenidos con división aleatoria tienden a sobrestimar significativamente la capacidad predictiva real del modelo.

#### Metodología de División Cronológica:

La división cronológica implementada respeta la naturaleza secuencial de los datos operacionales, utilizando un punto de corte temporal que separa:

- **Conjunto de Entrenamiento**: Datos históricos (80% inicial del dataset)
- **Conjunto de Prueba**: Datos más recientes (20% final del dataset)

Esta metodología simula fielmente el escenario operacional donde el modelo predice fallas futuras basándose únicamente en el historial de operación disponible hasta el momento de la predicción.

In [4]:
# Implementación de división cronológica de datos

print("=== Implementación de División Cronológica ===")

# Definición del punto de corte temporal (80% para entrenamiento)
train_size = 0.8
split_index = int(len(df) * train_size)

print(f"Tamaño total del dataset: {len(df)} muestras")
print(f"Punto de corte temporal: índice {split_index}")
print(f"Proporción de entrenamiento: {train_size*100}%")
print(f"Proporción de prueba: {(1-train_size)*100}%")

# División cronológica de características
X_train = X.iloc[:split_index].copy()
X_test = X.iloc[split_index:].copy()

# División cronológica de variable objetivo
y_train = y.iloc[:split_index].copy()
y_test = y.iloc[split_index:].copy()

print(f"\n=== Dimensiones de los Conjuntos Resultantes ===")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

# Análisis de distribución de clases en cada conjunto
print(f"\n=== Distribución de Clases por Conjunto ===")

train_distribution = y_train.value_counts().sort_index()
test_distribution = y_test.value_counts().sort_index()

print("Conjunto de Entrenamiento:")
print(f"  Clase 0 (normal): {train_distribution[0]} ({train_distribution[0]/len(y_train)*100:.2f}%)")
print(f"  Clase 1 (pre-falla): {train_distribution[1]} ({train_distribution[1]/len(y_train)*100:.2f}%)")
print(f"  Ratio de desbalance: {train_distribution[0]/train_distribution[1]:.1f}:1")

print("\nConjunto de Prueba:")
print(f"  Clase 0 (normal): {test_distribution[0]} ({test_distribution[0]/len(y_test)*100:.2f}%)")
print(f"  Clase 1 (pre-falla): {test_distribution[1]} ({test_distribution[1]/len(y_test)*100:.2f}%)")
print(f"  Ratio de desbalance: {test_distribution[0]/test_distribution[1]:.1f}:1")

# Visualización de la división cronológica
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Distribución temporal de la variable objetivo
ax1.plot(range(len(y)), y.values, alpha=0.7, linewidth=0.8)
ax1.axvline(x=split_index, color='red', linestyle='--', linewidth=2, label='Punto de División')
ax1.set_title('División Cronológica del Dataset')
ax1.set_xlabel('Índice Temporal')
ax1.set_ylabel('Variable Objetivo (falla)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Comparación de distribuciones de clases
sets = ['Entrenamiento', 'Prueba']
normal_counts = [train_distribution[0], test_distribution[0]]
failure_counts = [train_distribution[1], test_distribution[1]]

x = np.arange(len(sets))
width = 0.35

ax2.bar(x - width/2, normal_counts, width, label='Clase 0 (Normal)', alpha=0.8)
ax2.bar(x + width/2, failure_counts, width, label='Clase 1 (Pre-falla)', alpha=0.8)
ax2.set_title('Distribución de Clases por Conjunto')
ax2.set_xlabel('Conjunto de Datos')
ax2.set_ylabel('Número de Muestras')
ax2.set_xticks(x)
ax2.set_xticklabels(sets)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDivisión cronológica implementada correctamente")
print("El modelo será entrenado exclusivamente con datos históricos")
print("La evaluación se realizará sobre datos temporalmente posteriores")

=== Implementación de División Cronológica ===
Tamaño total del dataset: 19752 muestras
Punto de corte temporal: índice 15801
Proporción de entrenamiento: 80.0%
Proporción de prueba: 19.999999999999996%


NameError: name 'X' is not defined